In [1]:
from pathlib import Path
import tifffile as tp
from skimage import io, exposure, data, img_as_ubyte
from itertools import compress
import numpy as np
import os

In [2]:
def make_images(image_folder, 
                samples_list,
                output_folder,
                name_prefix='',
                minimum=0.2,
                max_quantile='q0.97',
                red=None,
                red_range=None,
                green=None,
                green_range=None,
                blue=None, 
                blue_range=None,
                magenta=None,
                magenta_range=None,
                cyan=None,
                cyan_range=None,
                yellow=None,
                yellow_range=None,
                white=None,
                white_range=None,
                roi_folder_save=False,
                simple_file_names=False,
                save_subfolder=''
                ):
    
    """This function will create RGB images using up to 7 channels, similar to MCD Viewer or ImageJ 
    Args:
        image_folder:
            The folder with the source images. In this folder, each ROI should have its own subfolder, with the folder named after the ROI.
        samples_list:
            List of samples to use from the image_folder. Only these samples will be used for auto-exposure if using quatiles (which is the default)
        output_folder:
            Output folder for images
        name_prefix:
            A prefix that will be put at the front of the file names
        minimum:
            The default minimum value (see 'range' below')
        max_quantile:
            The default max staining, which is the 97th quantile
        {colour}:
            The name of the marker to use for that colour
        {colour_range}:
            You can specify the range that should be the maximum and minimum, using the format (minimum, maximum). Both are specified by raw counts by default. However, if you put in a 'q' before a number, it will use that quantile range instead. e.g. 'q0.99' will use the 99th percentile as the maximum.
    Returns:
        Saves a .png per roi in the specified output directory
    """

    if not isinstance(samples_list, list):
        samples_list = [samples_list]
   
    # Create output directory if doesn't exist
    output = Path(output_folder)
    try:
        os.makedirs(output_folder)
    except:
        pass
    
    # Define empty lists
    red_imgs, green_imgs, blue_imgs,magenta_imgs,cyan_imgs,yellow_imgs,white_imgs = [], [], [], [], [], [], []
    red_rois, green_rois, blue_rois,magenta_rois,cyan_rois,yellow_rois,white_rois = [], [], [], [], [], [], []    
    roi_master=[]


    # Read in all the different colours (if used), including appropriate scaling
    # This is very sloppy and not very pythonic, but it works!

    if red is not None:
        
        if red_range is not None:
            min_v=red_range[0]
            max_q=red_range[1]
        else:
            min_v=minimum
            max_q=max_quantile
            
        red_imgs, red_rois = load_rescale_images(image_folder, samples_list, red, min_v, max_q)
        roi_master.append(red_rois)
        red="r_"+red+"_"

    if green is not None:
        
        if green_range is not None:
            min_v=green_range[0]
            max_q=green_range[1]
        else:
            min_v=minimum
            max_q=max_quantile
            
        green_imgs, green_rois = load_rescale_images(image_folder, samples_list, green, min_v, max_q)
        roi_master.append(green_rois)
        green='g_'+green+"_"
    
    if blue is not None:
        
        if blue_range is not None:
            min_v=blue_range[0]
            max_q=blue_range[1]
        else:
            min_v=minimum
            max_q=max_quantile
        
        blue_imgs, blue_rois = load_rescale_images(image_folder, samples_list, blue, min_v, max_q)
        roi_master.append(blue_rois)
        blue='b_'+blue+"_"
                
    if magenta is not None:
        
        if magenta_range is not None:
            min_v=magenta_range[0]
            max_q=magenta_range[1]
        else:
            min_v=minimum
            max_q=max_quantile
                       
        magenta_imgs, magenta_rois = load_rescale_images(image_folder, samples_list, magenta, min_v, max_q)
        roi_master.append(magenta_rois)
        magenta='m_'+magenta+"_"
        
    if cyan is not None:
        
        if cyan_range is not None:
            min_v=cyan_range[0]
            max_q=cyan_range[1]
        else:
            min_v=minimum
            max_q=max_quantile
        
        cyan_imgs, cyan_rois = load_rescale_images(image_folder, samples_list, cyan, min_v, max_q)
        roi_master.append(cyan_rois)
        cyan='c_'+cyan+"_"
       
    if yellow is not None:
        
        if yellow_range is not None:
            min_v=yellow_range[0]
            max_q=yellow_range[1]
        else:
            min_v=minimum
            max_q=max_quantile
        
        yellow_imgs, yellow_rois = load_rescale_images(image_folder, samples_list, yellow, min_v, max_q)
        roi_master.append(yellow_rois)
        yellow = 'y_'+yellow+"_"   

    if white is not None:
                        
        if white_range is not None:
            min_v=white_range[0]
            max_q=white_range[1]
        else:
            min_v=minimum
            max_q=max_quantile
            
        white_imgs, white_rois = load_rescale_images(image_folder, samples_list, white, min_v, max_q)
        roi_master.append(white_rois)
        white='w_'+white+"_"
        
    # Calculate number of ROIs found    
    num_rois = np.array([len(x) for x in [red_rois, green_rois, blue_rois,magenta_rois,cyan_rois,yellow_rois,white_rois]]).max()
    print(f'Found {num_rois} regions of interest')
    
    
    # Set non-used colours to empty
    if red is None:
        red_imgs = [0 for x in range(num_rois)]
        red=''
        
    if green is None:
        green_imgs = [0 for x in range(num_rois)]
        green=''
        
    if blue is None:
        blue_imgs = [0 for x in range(num_rois)]
        blue=''
        
    if magenta is None:
        magenta_imgs = [0 for x in range(num_rois)]
        magenta=''
        
    if cyan is None:
        cyan_imgs = [0 for x in range(num_rois)]
        cyan=''
        
    if yellow is None:
        yellow_imgs = [0 for x in range(num_rois)]
        yellow=''           
                
    if white is None:
        white_imgs = [0 for x in range(num_rois)]
        white=''      
    
    # Add up the various colours to make an RGB compatible colour space   
    red_summary = [np.clip((white_imgs[x] + red_imgs[x] + magenta_imgs[x] + yellow_imgs[x]),0,1) for x in range(num_rois)]
    blue_summary = [np.clip((white_imgs[x] + blue_imgs[x] + magenta_imgs[x] + cyan_imgs[x]),0,1) for x in range(num_rois)]
    green_summary = [np.clip((white_imgs[x] + green_imgs[x] + cyan_imgs[x] + yellow_imgs[x]),0,1) for x in range(num_rois)]
          
    
    # If using images which have no R, G or B at all, then use empty values
    for sample, r, g, b in zip(roi_master[0], red_summary, green_summary, blue_summary):
    
        print('Saving: '+sample)
        
        if np.shape(b)==():
            if np.shape(g)==():
                b=np.zeros(r.shape)
                g=np.zeros(r.shape)
            elif np.shape(r)==():
                b=np.zeros(g.shape)                
                r=np.zeros(g.shape)
            else:
                b=np.zeros(r.shape)
        elif np.shape(r)==():
            if np.shape(g)==():
                r=np.zeros(b.shape)
                g=np.zeros(b.shape)
            else:
                r=np.zeros(b.shape)
        elif np.shape(g)==():
            g=np.zeros(r.shape)
                  
        
        stack = np.dstack((r,g,b))
                
        # If using sample_file_names, then each image just gets saved as its ROI name
        if not simple_file_names:
            filename=f'{name_prefix}{sample}_{red}{green}{blue}{yellow}{cyan}{magenta}{white}'
            filename = filename.rstrip('_')
        else:
            filename=sample
        
        if not roi_folder_save:
            save_path=os.path.join(output_folder,f'{filename}.png')
        else:
            # Create output directory if doesn't exist
            
            roi_folder=os.path.join(output_folder,sample)           

            try:
                os.makedirs(roi_folder)
            except:
                pass
            
            save_path=os.path.join(roi_folder,f'{filename}.png')
        
        if save_subfolder!='':
             
            sub_path=os.path.join(output_folder,save_subfolder)           

            try:
                os.makedirs(sub_path)
            except:
                pass
            
            save_path=os.path.join(sub_path,f'{filename}.png')       
                    
        io.imsave(save_path,img_as_ubyte(stack))